In [1]:
##############
#Dependencies#
##############
%matplotlib notebook
import datetime
import pandas
import numpy as np
import folium


ModuleNotFoundError: No module named 'folium'

In [2]:
route = ['S567','S583','S568','S582','S569','S570','S581',
'S571','S579','S572','S578','S573',"S577","S587"]
station_df = pandas.read_csv('Data/station_config.csv')
route_df= pandas.read_csv('Data/route_summary.csv')
station_df = station_df.set_index("Station")
route_df = route_df.set_index("Station")
route_df= route_df.replace("UNKNOWN",0)
route_df= route_df.fillna(0)
route_df.head()

,Station.1,Time,Occupancy,Speed,Flow
Station,,,,,
S567,0.0,Wed Oct 03 01:16:59 CDT 2018,0.31,54,0
S567,0.0,Wed Oct 03 01:16:59 CDT 2018,0.31,54,0
S567,0.0,Sun Oct 07 21:21:29 CDT 2018,4.13,63,600
S583,0.0,Wed Oct 03 01:16:59 CDT 2018,0.24,55,0
S583,0.0,Wed Oct 03 01:16:59 CDT 2018,0.24,55,0


In [4]:
station_df.head()

,Unnamed: 0,Lat,Lng
Station,,,
no ID,0,44.00701,-92.48221
no ID,1,45.60591,-94.48746
no ID,2,45.60418,-94.48424
no ID,3,45.60406,-94.48413
no ID,4,45.60404,-94.48405


In [4]:

Results = {}


locations = []

for station in route:
        Occupancy =  float(route_df.loc[station,'Occupancy'].tail(1)[0])
        Speed = int(route_df.loc[station,'Speed'].tail(1)[0])

        Max_Occupancy = route_df.loc[station,'Occupancy'].astype(dtype="float")
        Max_Occupancy = max(Max_Occupancy.values)
        Max_Speed = route_df.loc[station,'Speed'].astype(dtype="int")
        Max_Speed = max(Max_Speed.values)
        
        lat, lon = station_df.loc[station, ['Lat', 'Lng']]
        locations.append((lat, lon))
    
        Results[station] = {
            "Current Speed" : Speed,
            "Current Occupancy" : Occupancy,
            "Max Speed" : Max_Speed,
            "Max Occupancy" : Max_Occupancy,
            "% of Max Speed" : round(Speed / Max_Speed * 100, 2),
            "% of Max Occupancy" : round(Occupancy / Max_Occupancy * 100, 2) ,
            "Lat":lat, 
            "Lon":lon}

In [5]:
Results = pandas.DataFrame(Results).T
Results.head()
Results

,% of Max Occupancy,% of Max Speed,Current Occupancy,Current Speed,Lat,Lon,Max Occupancy,Max Speed
S567,64.16,62.07,3.67,54.0,44.97447,-93.24870,5.72,87.0
S583,59.83,76.92,2.89,70.0,44.98175,-93.24397,4.83,91.0
S568,18.56,91.55,0.67,65.0,44.98399,-93.24197,3.61,71.0
S582,78.49,79.25,2.70,84.0,44.99115,-93.23659,3.44,106.0
S569,37.94,100.00,1.07,133.0,44.99116,-93.23629,2.82,133.0
S570,57.18,96.63,2.11,86.0,44.99644,-93.23823,3.69,89.0
S581,72.07,95.00,3.07,76.0,44.99646,-93.23853,4.26,80.0
S571,23.85,78.16,5.06,68.0,45.00199,-93.23465,21.22,87.0
S579,40.15,97.50,1.57,78.0,45.00091,-93.22346,3.91,80.0
S572,46.67,85.88,4.42,73.0,45.00070,-93.22349,9.47,85.0


In [6]:
folium_map = folium.Map(locations[5],
                        zoom_start=13,
                        tiles="CartoDB positron")
for index, row in Results.iterrows():

    
    folium.CircleMarker(location=(row["Lat"],
                                  row["Lon"]),
                        radius=1,
                        
                        fill=True).add_to(folium_map)

In [7]:
Features = []
Last_Sensor = []
for index, row in Results.iterrows():
    Current_Sensor = (row['Lat'], row['Lon'])
    if Last_Sensor == [] :
        Last_Sensor = (row['Lat'], row['Lon'])
    else:
    
        folium.PolyLine([Current_Sensor,Last_Sensor],
         weight=row['Current Occupancy'] * 2.5,
                        popup=f"Speed:{row['Current Speed']} Occupancy: {row['Current Occupancy']}").add_to(folium_map)

        Last_Sensor = (row['Lat'], row['Lon'])





In [8]:
folium_map


In [13]:
folium_map.save(f'results/maps/routemap{datetime.datetime.now().strftime("%b%d_%H%M%S")}.html')
print(f'Map saved at results/maps/routemap{datetime.datetime.now().strftime("%b%d_%H%M%S")}.html')

Map saved at results/maps/routemapOct07_075654.html
